# Model Editing

We use our IG and AP pipeline to localise important components. These components are edited using gradient descent to "unlearn" information. We evaluate our results on the CounterFact dataset.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
from torch import Tensor
import torch.optim as optim

from testing import logit_diff_metric
from applications.pipeline import run_attribution_steps, identify_target_components, optimise_edit_components, AttributionMethod, edit_model
from applications.datasets import CounterFact

from transformer_lens import HookedTransformer
from transformer_lens.utils import get_device
import copy

/vol/bitbucket/mwl21/fypvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = get_device()
# device = torch.device("cpu")
model = HookedTransformer.from_pretrained("gpt2-small", device=device)

# Explicitly calculate and expose the result for each attention head
model.set_use_attn_result(True)
model.set_use_hook_mlp_in(True)

Loaded pretrained model gpt2-small into HookedTransformer


## Editing procedure

In [5]:
# Verify that loading works, for one example
n_samples = 5

counterfact_dataset = CounterFact(model)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=1)

clean_input, corrupted_input, labels = next(iter(counterfact_dataloader))
# clean_input, corrupted_input, labels = counterfact_dataset.get_single_sample(0)

# print(clean_input)
# print(corrupted_input)
# print(labels)


In [6]:
from applications.pipeline import localise_models

counterfact_dataset = CounterFact(model)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=n_samples)

clean_input, corrupted_input, labels = next(iter(counterfact_dataloader))

target_mlp, target_attn = localise_models(model, clean_input, corrupted_input, labels, overwrite=False)

In [7]:
from applications.metrics import evaluate_counterfact_efficacy, evaluate_counterfact_paraphrased, evaluate_counterfact_neighborhood, evaluate_consistency
from applications.datasets import CounterFact
import pandas as pd
from collections import defaultdict

evaluation_scores = defaultdict(list)
counterfact_dataset = CounterFact(model)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=1)

for n, (clean_input, corrupted_input, labels) in enumerate(counterfact_dataloader):

    original_output = model.generate(clean_input, max_new_tokens=5, do_sample=False)

    edited_model = edit_model(model, clean_input, corrupted_input, labels, target_mlp[n], target_attn[n])

    # Evaluate
    print(f"Prompt: {clean_input}")
    print("Original output:", original_output)
    # print(f"Original answer: {labels[:, 0]}. Target answer: {labels[:, 1]}")

    print(edited_model.generate(clean_input, max_new_tokens=5, do_sample=False))

    score, magnitude = evaluate_counterfact_efficacy(edited_model, n, verbose=True)
    evaluation_scores["Efficacy score"].append(score.item())
    evaluation_scores["Efficacy magnitude"].append(magnitude.item())

    score, magnitude = evaluate_counterfact_paraphrased(edited_model, n, verbose=False)
    evaluation_scores["Generalisation score"].append(score.item())
    evaluation_scores["Generalisation magnitude"].append(magnitude.item())

    score, magnitude = evaluate_counterfact_neighborhood(edited_model, n, verbose=False)
    evaluation_scores["Specificity score"].append(score.item())
    evaluation_scores["Specificity magnitude"].append(magnitude.item())

    consistency_score = evaluate_consistency(model, n, verbose=False)
    evaluation_scores["Consistency score"].append(score.item())
    evaluation_scores["Consistency magnitude"].append(magnitude.item())

    if n + 1 >= n_samples: break

evaluation_df = pd.DataFrame(evaluation_scores)
evaluation_df.to_csv('results/counterfact/evaluation.csv')

100%|██████████| 5/5 [00:00<00:00, 10.03it/s]



Fine tuning model...
Target MLP tensor(656, device='cuda:0')
Target attn tensor(17, device='cuda:0')
Total loss: 13.334612846374512, forget loss: 0.8218996524810791, rewrite loss: 12.944661140441895, fluency loss: 0.3899519145488739
Total loss: 9.082454681396484, forget loss: 0.8529210686683655, rewrite loss: 8.922614097595215, fluency loss: 0.15984010696411133
Total loss: 6.586724758148193, forget loss: 0.3131824731826782, rewrite loss: 6.527433395385742, fluency loss: 0.059291377663612366
Total loss: 3.711986541748047, forget loss: 0.5813448429107666, rewrite loss: 3.5793399810791016, fluency loss: 0.1326465755701065
Total loss: 0.457078218460083, forget loss: 0.5283988118171692, rewrite loss: 0.2916354537010193, fluency loss: 0.16544277966022491
Prompt: ['The mother tongue of Danielle Darrieux is']
Original output: The mother tongue of Danielle Darrieux is French.

The


100%|██████████| 5/5 [00:00<00:00, 59.27it/s]

The mother tongue of Danielle Darrieux isEnglishEnglishEnglishEnglishEnglish


Original label: French
Target label: English
Outputs: ['English', ' the', 'English', ' New', 'English', 'English', ' New', ' the', ' New', ' New']


...


Fine tuning model...
Target MLP tensor(946, device='cuda:0')
Target attn tensor(6, device='cuda:0')
Total loss: 19.299882888793945, forget loss: 0.8302270174026489, rewrite loss: 19.210573196411133, fluency loss: 0.08931056410074234
Total loss: 14.409964561462402, forget loss: 0.8936126232147217, rewrite loss: 14.318363189697266, fluency loss: 0.09160180389881134
Total loss: 10.669249534606934, forget loss: 0.9181691408157349, rewrite loss: 10.54464340209961, fluency loss: 0.12460607290267944
Total loss: 7.067913055419922, forget loss: 0.9471347332000732, rewrite loss: 6.956719398498535, fluency loss: 0.11119356751441956
Total loss: 3.53542423248291, forget loss: 0.9746201038360596, rewrite loss: 3.4117677211761475, fluency loss: 0.12365640699863434
Total loss: 1.0128750801086426, forget loss: 0.7791879177093506, rewrite loss: 0.8196882009506226, fluency loss: 0.19318681955337524
Total loss: 0.0946325734257698, forget loss: 0.13005399703979492, rewrite loss: 0.04287965968251228, fluen

100%|██████████| 5/5 [00:00<00:00, 58.73it/s]

What is the twin city of Lyon? It isManila,Manila


Original label: Beirut
Target label: Manila
Outputs: [' its', ' its', ' the', ' the', ' the', ' the', ' its', ' the', ' the', ' the']


100%|██████████| 5/5 [00:00<00:00, 48.34it/s]


## Evaluation

For each sample, we calculate the efficacy, generalisability, specificity and consistency for:

- The original models' outputs
- The edited model's outputs


In [9]:
evaluation_df.head()

,Efficacy score,Efficacy magnitude,Generalisation score,Generalisation magnitude,Specificity score,Specificity magnitude,Consistency score,Consistency magnitude
0,1.0,0.000750,1.0,0.000212,1.0,0.000581,1.0,0.000581
1,1.0,0.000052,1.0,0.000380,1.0,0.100113,1.0,0.100113
2,1.0,0.002326,1.0,0.003862,1.0,0.005302,1.0,0.005302
3,1.0,0.001120,1.0,0.182638,1.0,0.047355,1.0,0.047355
4,0.6,0.000243,1.0,0.000471,1.0,0.200048,1.0,0.200048
